In [1]:
#reference
from konlpy.tag import Okt
from collections import Counter
import re

In [2]:
#topic class
class topic(object):
    def get_freq(self, data):
        pocket_size = 20
        noun = []
        noun_proto = okt.nouns(data)
        
        for words in noun_proto:
            if (len(words)>1) and (words not in stop_words):
                noun.append(words)
                
        count = Counter(noun)        
        noun_list = count.most_common(pocket_size)
    
        if(len(noun_list) < pocket_size):
            pocket_size = len(noun_list)
        
        return noun_list
    
    def get_topic(self, data, amount):
        doc = []
        for article in data:
            freq_list = self.get_freq(article)
            
            for freq in freq_list:
                doc.append((freq[0]))
            
        total = Counter(doc)
    
        return total.most_common(amount)
    
    # 특정 keyword 를 포함하는 기사만 활용해 topic 추출 > 연관단어 추출
    def get_topic_include(self, data, amount, keyword):
        doc = []
        for article in data:
            if(okt.nouns(article).count(keyword) == 0):continue
                
            freq_list = self.get_freq(article)

            for freq in freq_list:
                doc.append((freq[0]))
            
        total = Counter(doc)
    
        return total.most_common(amount)

In [3]:
#weight calculater
class calc(object):
    def __init__(self, data):
        self.data = data
        
    def cal_weight(self, topic_words):
        freq_sum = 0
        result_sum = 0
        result_list = []
        for words in topic_words: #핵심단어로 지정된 단어들이 등장한 문서의 합
            freq_sum+= words[1]

        for article in self.data:
            result_sum = 0
            for words in topic_words:
                result_sum += words[1] * (okt.nouns(article).count(words[0])/len(article))
                #result_sum += (words[1]/freq_sum) * (okt.nouns(article).count(words[0])/len(article)) 잠시변경
            result_list.append(result_sum)
        return result_list

In [4]:
#result
def result_generator(selec):
    max_ = 0
    index_ = 0
    include = topics.get_topic_include(data_split, 10, main_topic[selec][0])
    calculater = calc(data_split)
    result = calculater.cal_weight(include)
    i = 1
    print(include[0][0],"에 대해서")
    print(include)
    for value in result:
        print(i,"번째기사 =", value)
        if(value > max_):
            max_ = value
            index_ = i
        i+=1
    print("max:", index_,"번째기사 =", max_)
    

In [5]:
#main frame 1 - 데이터 로드 및 선언
okt = Okt()
doc = []
topics = topic()

#stop-words load
fp = open("stopwords.txt", "r", encoding = 'UTF8')
stop_words = fp.read().split()

#테스트용
fp = open("input.txt", "r", encoding = 'UTF8')
data = fp.read()
data_split = data.split("\n\n")
fp.close()#

#실제
#data = news_collector.collect()
#data_split = data.split("\n\n")

In [6]:
#main frame 2 - 전체 데이터(기사)의 토픽 > 핫이슈
main_topic = topics.get_topic(data_split, 5)
print(main_topic)

[('코로나', 5), ('정부', 4), ('경기도', 3), ('창고', 3), ('사람', 3)]


In [7]:
#main frame 3 - 핫이슈 별 연관 키워드(해당 단어를 포함한 기사들만 모아서 재분석)
for issue in main_topic:
    include = topics.get_topic_include(data_split, 10, issue[0])
    print("<Topic:", issue[0],">")
    print(include, "\n")

<Topic: 코로나 >
[('코로나', 5), ('정부', 4), ('경기도', 3), ('창고', 3), ('사람', 3), ('백신', 3), ('정도', 3), ('경찰', 3), ('새벽', 3), ('가격', 3)] 

<Topic: 정부 >
[('정부', 4), ('중국', 2), ('코로나', 2), ('미국', 1), ('증상', 1), ('쿠바', 1), ('직원', 1), ('괴질', 1), ('원인', 1), ('공격', 1)] 

<Topic: 경기도 >
[('경기도', 3), ('시흥시', 2), ('정왕동', 2), ('창고', 2), ('진화', 2), ('작업', 2), ('불로', 2), ('소방', 2), ('코로나', 2), ('백신', 2)] 

<Topic: 창고 >
[('창고', 3), ('경기도', 2), ('시흥시', 2), ('정왕동', 2), ('진화', 2), ('작업', 2), ('불로', 2), ('소방', 2), ('코로나', 2), ('물류', 1)] 

<Topic: 사람 >
[('사람', 3), ('정도', 3), ('정부', 2), ('경찰', 2), ('새벽', 2), ('인물', 2), ('진술', 2), ('실종', 2), ('친구', 2), ('낚시꾼', 2)] 



In [ ]:
#test1_사용자 입력
status = True
j = 0
for words in main_topic:
    print((j+1),":", words[0], sep ='', end = ' ')
    j += 1
print("0:종료")

while(status):
    print("입력:",end = '')
    sel = int(input())-1 
    if(sel == 0):result_generator(sel)
    elif(sel == 1):result_generator(sel)
    elif(sel == 2):result_generator(sel)
    elif(sel == 3):result_generator(sel)
    elif(sel == 4):result_generator(sel)
    elif(sel == 0):status = False
    else:print("입력이 잘못되었습니다")

1:코로나 2:정부 3:경기도 4:창고 5:사람 0:종료
입력:4
창고 에 대해서
[('창고', 3), ('경기도', 2), ('시흥시', 2), ('정왕동', 2), ('진화', 2), ('작업', 2), ('불로', 2), ('소방', 2), ('코로나', 2), ('물류', 1)]
1 번째기사 = 0.0019083969465648854
2 번째기사 = 0.0014134275618374558
3 번째기사 = 0.111731843575419
4 번째기사 = 0.0019743336623889436
5 번째기사 = 0.0021164021164021165
6 번째기사 = 0.0013123359580052493
7 번째기사 = 0.0025906735751295338
8 번째기사 = 0.02113022113022113
9 번째기사 = 0.0830188679245283
10 번째기사 = 0.002081165452653486
11 번째기사 = 0.0148975791433892
12 번째기사 = 0.000946969696969697
13 번째기사 = 0.0032626427406199023
14 번째기사 = 0.010638297872340425
15 번째기사 = 0.0057859209257473485
16 번째기사 = 0.004705882352941176
17 번째기사 = 0.006060606060606061
18 번째기사 = 0.0006851661527920521
max: 3 번째기사 = 0.111731843575419
입력: